In [1]:
# Here we import what we need for the following code 
import tensorflow as tf
from PIL import Image 
from random import * 
import numpy as np

In [ ]:
#We call the training fonction which gives us the data preprocessed and the model trained 
%run training.ipynb

In [ ]:
# Definition of the function applying the Project Gradient Descent attack method 
def pgd_attack(model, img, labels, eps, alpha, num_iter):
    img_disturbed = img
    for _ in range(num_iter):
        with tf.GradientTape() as t:
            t.watch(img_disturbed)
            predictions = model(img_disturbed)
            loss = SparseCategoricalCrossentropy(from_logits=True)(labels, predictions)

        gradients = t.gradient(loss, img_disturbed)
        img_disturbed = img_disturbed + alpha * tf.sign(gradients)
        img_disturbed = tf.clip_by_value(img_disturbed, img - epsilon, img + epsilon)
        img_disturbed = tf.clip_by_value(img_disturbed, 0, 1)
    return img_disturbed

# We set the parameters
epsilon = 0.1  
alpha = 0.01 
num_iter = 40  

#we separate labels and images from our test set
for i in range(len(test_set)):
    test_images.append(test_set[i][0])
    test_labels.append(int(test_set[i][1][-2:]))
    
test_images = np.array(test_images)
test_labels = np.array(test_labels)

#we apply the PGD to our test set 
test_img_tensors = tf.convert_to_tensor(test_images) #convert images into tensors
adversarial_img = pgd_attack(model, test_img_tensors, test_labels, epsilon, alpha, num_iter)

# We now test the efficiency of the adversarial examples we created (How much it impacted the accuracy of the model) 
test_loss, test_acc = model.evaluate(adversarial_img, test_labels, verbose=2)
print('\nTest accuracy on adversarial images:', test_acc)

In [ ]:
#here is a comparison of a random image before and after the perturbation
r = randint(0,len(test_set)) 
tf.keras.utils.array_to_img(test_image[0])
tf.keras.utils.array_to_img(img_adversarial[0])